# Neural Network with Quadratic Data
- y = x^2 + 3
- 4 hidden layers

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom() 

Setting up a new session...


### Data Generation

In [12]:
num_data = 1000
num_epoch = 5000

noise = init.normal_(torch.FloatTensor(num_data,1), std=3)
x = init.uniform_(torch.Tensor(num_data,1), -15, 15)
y = x**2 + 3
y_noise = y + noise

In [13]:
input_data = torch.cat([x,y_noise],1) 
win = viz.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-15,
        xtickmax=15,
        xtickstep=1,
        ytickmin=0,
        ytickmax=230,
        ytickstep=10,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=3,
    ),
)

In [14]:
origin_data = torch.cat([x, y], 1)
win = viz.scatter(X = origin_data, win=win, update='append')

### Model & Optimizer

In [15]:
# FCN with 4 hidden layers

model = nn.Sequential(
            nn.Linear(1,6),
            nn.ReLU(),
            nn.Linear(6,10),
            nn.ReLU(),
            nn.Linear(10,6),
            nn.ReLU(),
            nn.Linear(6,1)
        ).cuda()
loss_func = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.0005) 

### Train

In [17]:
loss_arr = []
label = Variable(y_noise.cuda())

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x.cuda()))
    
    loss = loss_func(output, label)
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('epoch No', i, loss)
    loss_arr.append(loss.cpu().data.numpy()) 

epoch No 0 tensor(78.6847, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 100 tensor(78.4766, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 200 tensor(78.2514, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 300 tensor(77.9680, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 400 tensor(77.5523, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 500 tensor(76.8322, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 600 tensor(75.2945, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 700 tensor(70.7346, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 800 tensor(43.3843, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 900 tensor(19.3564, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 1000 tensor(19.0211, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 1100 tensor(18.6786, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 1200 tensor(18.3201, device='cuda:0', grad_fn=<L1LossBackward>)
epoch No 1300 tensor(17.9386, device='cuda:0', grad_fn=<L1LossB

In [18]:
param_list = list(model.parameters())
print(param_list) 

[Parameter containing:
tensor([[-0.1816],
        [ 0.6914],
        [ 0.0862],
        [-0.8359],
        [-0.1469],
        [-0.2567]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-1.5279, -2.1387, -0.7578, -0.7833, -1.4511, -1.8160], device='cuda:0',
       requires_grad=True), Parameter containing:
tensor([[ 0.3170,  0.5966, -0.1891,  0.3047,  0.2720, -0.3270],
        [ 0.6646,  1.3024,  0.0720,  0.3693,  0.8643,  0.6753],
        [ 0.8588,  1.1702,  0.2778,  0.7369,  0.6208,  0.4772],
        [-0.1771,  0.0081, -0.2661,  0.0847,  0.0703, -0.2567],
        [ 0.0742,  0.4681,  0.0921, -0.1542,  0.0085, -0.2248],
        [-0.0189, -0.1562, -0.3858,  0.2143,  0.2526, -0.2745],
        [ 0.2466, -0.0247, -0.0374, -0.4023, -0.3548, -0.2375],
        [ 0.3135,  0.6441, -0.1065,  0.0588,  0.1293,  0.5769],
        [-0.2991, -0.3974, -0.2622, -0.1994, -0.1443, -0.3838],
        [ 0.4492,  0.7102,  0.6890,  0.6102,  0.6239,  1.0386]],
       device='cuda:0', require

### Visualize Trained Output

In [20]:
win2 = viz.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-15,
        xtickmax=15,
        xtickstep=1,
        ytickmin=0,
        ytickmax=230,
        ytickstep=10,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=3,
    ),
) 

In [22]:
output_data = torch.cat([x, output.cpu().data], 1)
win = viz.scatter(X = output_data, win=win2, update='append') 

### Visualize Loss Graph

In [23]:
x = np.reshape([i for i in range(num_epoch)], newshape=[num_epoch])
loss_data = np.reshape(loss_arr, newshape=[num_epoch])

win3 = viz.line(X = x, Y = loss_data) 